In [5]:
import pandas as pd
import numpy as np
from icecream import ic
'''
 - (1) pivot(), pd.pivot_table()
 90도 회전(rotation)
 칼럼 -> 인덱스로 변환

 - (2) stack(), unstack()
stack이 (위에서 아래로 길게, 높게) 쌓는 것이면
unstack은 쌓은 것을 옆으로 늘어놓는것(왼쪽에서 오른쪽으로 넓게) 라고 연상이 될 것입니다.

 - (3) melt()

 - (4) wide_to_long()

 - (5) pd.crosstab()
'''
# 30 다음 객체를 customer_id 를 인덱스로하고 product_code 를 컬럼으로, purchare_amount 를 값으로, 재구성하시오

df = pd.DataFrame({"customer_id":['kim','lee','park','song','yoon','kang','tak','ryu','jang'],
               "product_code":['com','phone','tv','com','phone','tv','com','phone','tv'],
               "grade":['A','A','A','A','A','A','B','B','B'],
               "purchase_amount":[30,10,0,40,15,30,0,0,10]})
ic(df)

df = df.pivot(index='customer_id', columns='product_code', values='purchase_amount')
ic(df)

ic| df:   customer_id product_code grade  purchase_amount
        0         kim          com     A               30
        1         lee        phone     A               10
        2        park           tv     A                0
        3        song          com     A               40
        4        yoon        phone     A               15
        5        kang           tv     A               30
        6         tak          com     B                0
        7         ryu        phone     B                0
        8        jang           tv     B               10
ic| df: product_code   com  phone    tv
        customer_id                    
        jang           NaN    NaN  10.0
        kang           NaN    NaN  30.0
        kim           30.0    NaN   NaN
        lee            NaN   10.0   NaN
        park           NaN    NaN   0.0
        ryu            NaN    0.0   NaN
        song          40.0    NaN   NaN
        tak            0.0    NaN   NaN
        yoon        

product_code,com,phone,tv
customer_id,,,
jang,NaN,NaN,10.0
kang,NaN,NaN,30.0
kim,30.0,NaN,NaN
lee,NaN,10.0,NaN
park,NaN,NaN,0.0
ryu,NaN,0.0,NaN
song,40.0,NaN,NaN
tak,0.0,NaN,NaN
yoon,NaN,15.0,NaN


In [6]:
# 31. 다음 객체를 customer_id, grade 를 인덱스로하고 product_code 를 컬럼으로, purchare_amount 를 값으로, 재구성하시오
df = pd.DataFrame({"customer_id":['kim','lee','park','song','yoon','kang','tak','ryu','jang'],
               "product_code":['com','phone','tv','com','phone','tv','com','phone','tv'],
               "grade":['A','A','A','A','A','A','B','B','B'],
               "purchase_amount":[30,10,0,40,15,30,0,0,10]})
df.pivot(index=['customer_id', 'grade'], columns='product_code', values='purchase_amount')

,product_code,com,phone,tv
customer_id,grade,,,
jang,B,NaN,NaN,10.0
kang,A,NaN,NaN,30.0
kim,A,30.0,NaN,NaN
lee,A,NaN,10.0,NaN
park,A,NaN,NaN,0.0
ryu,B,NaN,0.0,NaN
song,A,40.0,NaN,NaN
tak,B,0.0,NaN,NaN
yoon,A,NaN,15.0,NaN


In [17]:
# 32
df = pd.DataFrame(data=np.arange(16).reshape(4,4),
                  index=pd.MultiIndex.from_tuples([('kim', '2019'),('kim', '2020'),('lee','2019'),('lee', '2020')]),
                  columns= ['com', 'phone', 'tv', 'notebook'])
ic(df)

ic| df:           com  phone  tv  notebook
        kim 2019    0      1   2         3
            2020    4      5   6         7
        lee 2019    8      9  10        11
            2020   12     13  14        15


com  phone  tv  notebook
kim 2019    0      1   2         3
    2020    4      5   6         7
lee 2019    8      9  10        11
    2020   12     13  14        15

In [32]:
# ic(type(df.stack())) pandas.core.series.Series 스택으로 호출한 메소드는 series
# 칼럼의 level은 1개 밖에 없으므로 stack(level=-1) 을 별도로 명기하지 않아도 됨
df = df.stack(level=-1, dropna=True)
ic(df)
df = df.unstack(level=0)
ic(df.unstack())

'''
결측값이 있는 데이터셋을 stack() 할 때 결측값을 제거할지(dropna=True),
아니면 결측값을 NaN으로 유지할지(dropna=False) 설정
unstack(level=-1), unstack(level=0), unstack(level=1) 별로
어떤 level이 칼럼으로 이동해서 unstack() 되는지 체크
'''

df = df.reset_index()
df.rename(columns={'level_0':'customer', 'level_1':'year'}, inplace=True)
ic(df)


ic| df: 0  customer_id          kim
           product_code         com
           purchase_count         1
           purchase_amount      100
        1  customer_id          kim
           product_code       phone
           purchase_count         2
           purchase_amount      200
        2  customer_id          lee
           product_code          tv
           purchase_count         3
           purchase_amount      300
        3  customer_id          lee
           product_code         tab
           purchase_count         4
           purchase_amount      400
        dtype: object
ic| df.unstack(): 0  customer_id          kim
                     product_code         com
                     purchase_count         1
                     purchase_amount      100
                  1  customer_id          kim
                     product_code       phone
                     purchase_count         2
                     purchase_amount      200
                  2  customer_id  

,index,0,1,2,3
0,customer_id,kim,kim,lee,lee
1,product_code,com,phone,tv,tab
2,purchase_count,1,2,3,4
3,purchase_amount,100,200,300,400


In [36]:
'''
melt() 는 ID 변수를 기준으로 원래 데이터셋에 있던 여러개의 칼럼 이름을
'variable' 칼럼에 위에서 아래로 길게 쌓아놓고,
'value' 칼럼에 ID와 variable에 해당하는 값을
넣어주는 식으로 데이터를 재구조화
'''

df = pd.DataFrame({"customer_id":['kim','kim','lee','lee'],
                   "product_code":['com','phone','tv','tab'],
                   "purchase_count":[1,2,3,4],
                   "purchase_amount":[100,200,300,400]})
ic(df)

# (1) pd.melt(data, id_vars=['id1', 'id2', ...]) 를 사용한 데이터 재구조화
df = pd.melt(df, id_vars=['customer_id','product_code'])
ic(df)
# (2) pd.melt() 의 variable 이름, value 이름 부여하기 : var_name, value_name

ic| df:   customer_id product_code  purchase_count  purchase_amount
        0         kim          com               1              100
        1         kim        phone               2              200
        2         lee           tv               3              300
        3         lee          tab               4              400
ic| df:   customer_id product_code         variable  value
        0         kim          com   purchase_count      1
        1         kim        phone   purchase_count      2
        2         lee           tv   purchase_count      3
        3         lee          tab   purchase_count      4
        4         kim          com  purchase_amount    100
        5         kim        phone  purchase_amount    200
        6         lee           tv  purchase_amount    300
        7         lee          tab  purchase_amount    400


,customer_id,product_code,variable,value
0,kim,com,purchase_count,1
1,kim,phone,purchase_count,2
2,lee,tv,purchase_count,3
3,lee,tab,purchase_count,4
4,kim,com,purchase_amount,100
5,kim,phone,purchase_amount,200
6,lee,tv,purchase_amount,300
7,lee,tab,purchase_amount,400


In [40]:
import pandas as pd
import numpy as np
from icecream import ic

np.random.seed(10)
df = pd.DataFrame({"Class1":{0:"A",1:"B",2:"C"},
                  "Class2":{0:"D",1:"E",2:"F"},
                  "Score1":{0:"2.5",1:"1.2",2:"0.7"},
                  "Score2":{0:"3.2",1:"1.3",2:"0.1"},
                   "value": dict(zip(range(3),np.random.randn(3)))})
df["seq"] = df.index
# ic(df)
'''
pd.wide_to_long() 함수를 써서 데이터를 재구조화
wide_to_long()은 pivot() 이나 stack() 과는 다르게
"칼럼 이름의 앞부분"과 나머지 "칼럼 이름의 뒷부분"을 구분해서,
칼럼 이름의 앞부분을 칼럼 이름으로,
칼럼 이름의 나머지 뒷부분을 행(row)의 원소로 해서
세로로 길게(long~) 쌓아 줍니다.
pd.widt_to_long() 함수를 한번 사용해서
# 가로로 넓은 데이터(wide~)를 세로로 길게(long~) 재구조화 하시오.
pd.wide_to_long(data, ["col_prefix_1", "col_prefix_2"], i="idx_1", j="idx_2")
'''
data_long = pd.wide_to_long(df, ["Class", "Score"], i="seq", j="t")
ic(data_long)
'''
###
wide_to_long(DF, stubnames=[])
stub은 그루터기로 전체값에서 일부분의 값을 의미 Class1, Class2, Class3에서
반복되는 Class와 store라는 값만 취한다
df, ["Class-1", 'Class-2"]
i 는 DF의 인덱스로 정한 값을 입력
j는 stubname 에서 컬럼명으로정의한 나머지 부분은 코딩하는 사람의 임의로 정의
sep는 구분자로 stubname과 임의의 컬럼명 사이를 구분하는 패턴
sep에서 ''이면, 구분자가 없는 상태이고, 예제처럼 띄어쓰기 하려면 ' '로 줌
'''

ic| data_long:           value Class Score
               seq t                      
               0   1  1.331587     A   2.5
               1   1  0.715279     B   1.2
               2   1 -1.545400     C   0.7
               0   2  1.331587     D   3.2
               1   2  0.715279     E   1.3
               2   2 -1.545400     F   0.1


'\n###\nwide_to_long(DF, stubnames=[])\nstub은 그루터기로 전체값에서 일부분의 값을 의미 Class1, Class2, Class3에서\n반복되는 Class와 store라는 값만 취한다\ndf, ["Class-1", \'Class-2"]\ni 는 DF의 인덱스로 정한 값을 입력\nj는 stubname 에서 컬럼명으로정의한 나머지 부분은 코딩하는 사람의 임의로 정의\nsep는 구분자로 stubname과 임의의 컬럼명 사이를 구분하는 패턴\nsep에서 \'\'이면, 구분자가 없는 상태이고, 예제처럼 띄어쓰기 하려면 \' \'로 줌\n'

In [124]:
|'''
pd.crosstab()
범주형 변수로 되어있는 요인(factors)별로 교차분석(cross tabulations) 해서,
행, 열 요인 기준 별로 빈도를 세어서 도수분포표(frequency table),
교차표(contingency table) 를 생성한다

# 범주형 변수란 고유한 값이나 범주 수가 제한된 변수(예: 성별 또는 종교)이다.
# 교차분석(cross-tabulation analysis)은 '범주형'으로 구성된 자료들 간의 연관관계를 확인하기 위해
   교차표를 만들어 관계를 확인하는 분석 방법이다.
'''
df = pd.DataFrame({"학생ID" :["kim","kim","kim","lee","lee","park"],
                   "1학기학점":["A","A","A","B","B","B"],
                   "2학기학점":["D","D","D","C","C","D"]})
ic(df)

# (1) 교차표(contingency table, frequency table) 만들기 : pd.crosstab(index, columns)
df = pd.crosstab(df["1학기학점"] ,df["2학기학점"])
ic(df)
# (2) Multi-index, Multi-level로 교차표 만들기 : pd.crosstab([id1, id2], [col1, col2])

ic| df:    학생ID 1학기학점 2학기학점
        0   kim     A     D
        1   kim     A     D
        2   kim     A     D
        3   lee     B     C
        4   lee     B     C
        5  park     B     D
ic| df: 2학기학점  C  D
        1학기학점      
        A      0  3
        B      2  1


2학기학점,C,D
1학기학점,,
A,0,3
B,2,1


In [139]:
df = pd.DataFrame({"학생ID" :["kim","kim","kim","lee","lee","park"],
                   "1학기학점":["A","A","A","B","B","B"],
                   "2학기학점":["D","D","D","C","C","D"]})
# pd.crosstab(df['학생ID'], [df['1학기학점'], df['2학기학점']])

ic| df:    학생ID 1학기학점 2학기학점
        0   kim     A     D
        1   kim     A     D
        2   kim     A     D
        3   lee     B     C
        4   lee     B     C
        5  park     B     D


,학생ID,1학기학점,2학기학점
0,kim,A,D
1,kim,A,D
2,kim,A,D
3,lee,B,C
4,lee,B,C
5,park,B,D


In [8]:
df = pd.DataFrame({"학생ID" :["kim","kim","kim","lee","lee","park"],
                   "1학기학점":["A","A","A","B","B","B"],
                   "2학기학점":["D","D","D","C","C","D"]})
pd.crosstab(df['학생ID'], [df['1학기학점'], df['2학기학점']])

1학기학점  A  B   
2학기학점  D  C  D
학생ID          
kim    3  0  0
lee    0  2  0
park   0  0  1

In [11]:
# (3) 교차표의 행 이름, 열 이름 부여 : pd.crosstab(rownames=['xx'], colnames=['aa'])
import pandas as pd
df = pd.DataFrame({"학생ID" :["kim","kim","kim","lee","lee","park"],
                   "1학기학점":["A","A","A","B","B","B"],
                   "2학기학점":["D","D","D","C","C","D"]})
pd.crosstab(df["학생ID"],[df["1학기학점"], df["2학기학점"]],rownames=['id num'],colnames=['a_b', 'c_d'])

a_b     A  B   
c_d     D  C  D
id num         
kim     3  0  0
lee     0  2  0
park    0  0  1

In [27]:
# (4) 교차표의 행 합, 열 합 추가하기 : pd.crosstab(margins=True)


# (5) 구성비율로 교차표 만들기 : pd.crosstab(normalize=True)

KeyError: "None of [Index(['seq'], dtype='object')] are in the [columns]"